In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directoryz
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
!pip install --quiet transformers[torch] gdown datasets

In [22]:
# import gdown

# gdown.download("https://drive.google.com/uc?id=1DcRYrn_ZJwTdzXPjWNImO-mbHE0yRt8P", "data.csv")

In [23]:
# import pandas as pd

# df = pd.read_csv("data.csv")
# df = df.drop(['Unnamed: 0'], axis=1)
# df.head()

In [24]:
# df['tags'] = df['tags'].apply(lambda x: x[1:-1].replace("'", ''))

In [25]:
# df['text'] = "Write an essay on title: " + df['title'] + "with tags: " +df['tags'] + " \nArticle Text:  " + df['text']

In [26]:
# df_sample = df.sample(50000)

In [27]:
# df_sample = df_sample.reset_index().drop(['index'], axis=1)

In [28]:
# df_sample.dropna(inplace=True)

In [29]:
# df_sample = df_sample.astype(str)

In [30]:
# df_sample.head()

In [31]:
# sample_dir = "medium_sample.csv"

In [32]:
# df_sample.to_csv(sample_dir)

In [33]:
# from IPython.display import FileLink

# FileLink('medium_sample.csv')

In [34]:
import gdown

url = "https://drive.google.com/uc?id=1Nmu_Mq1tycGt9KUKvzsQTGoEr7007I96"
sample_dir = "medium_sample.csv"
gdown.download(url, "medium_sample.csv")

Downloading...
From (uriginal): https://drive.google.com/uc?id=1Nmu_Mq1tycGt9KUKvzsQTGoEr7007I96
From (redirected): https://drive.google.com/uc?id=1Nmu_Mq1tycGt9KUKvzsQTGoEr7007I96&confirm=t&uuid=c3de9425-1fc5-441d-afd6-c6a869c8cf2d
To: /kaggle/working/medium_sample.csv
100%|██████████| 271M/271M [00:03<00:00, 78.1MB/s] 


'medium_sample.csv'

In [35]:
model_name = "gpt2"
# data_path = "/content/drive/MyDrive/data/medium_articles_cleaned.csv"
output_dir = "model_gpt2"

In [36]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained(
    model_name,
    bos_token="<|startoftext|>",
    eos_token="<|endoftext|>",
    pad_token="<|pad|>"
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
print("The max model length is {} for this model, although the actual embedding size for GPT small is 768".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))


The max model length is 1024 for this model, although the actual embedding size for GPT small is 768
The beginning of sequence token <|startoftext|> token has the id 50257
The end of sequence token <|endoftext|> has the id 50256
The padding token <|pad|> has the id 50258


In [38]:
from datasets import load_dataset

data_files = {}
dataset_args = {}
validation_per = 5
data_files = {
    "train": sample_dir
}

raw_datasets = load_dataset(
    'csv',
    sep=',',
    data_files=data_files
)

raw_datasets['validation'] = load_dataset(
    'csv',
    sep=',',
    data_files=data_files,
    split=f"train[:{validation_per}%]",
    **dataset_args
)

raw_datasets['train'] = load_dataset(
    'csv',
    sep=',',
    data_files=data_files,
    split=f"train[{validation_per}%:]",
    **dataset_args
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-434e0e10b1eac58b/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
text_column_name = "text"
column_names = raw_datasets['train'].column_names 

preprocessing_num_workers = 3

def tokenize_function(examples):
  output = tokenizer(examples[text_column_name], truncation=True)
  return output

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    num_proc=preprocessing_num_workers,
    remove_columns=column_names,
    desc="Running tokenizer on dataset"
)

Running tokenizer on dataset #0:   0%|          | 0/16 [00:00<?, ?ba/s]

Running tokenizer on dataset #1:   0%|          | 0/16 [00:00<?, ?ba/s]

Running tokenizer on dataset #2:   0%|          | 0/16 [00:00<?, ?ba/s]

Running tokenizer on dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
from itertools import chain

block_size = 512

overwrite_cache = False

def group_texts(examples):
  concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])

  if total_length >= block_size:
    total_length = (total_length // block_size) * block_size

  result = {
      k: [t[i:i+block_size] for i in range(0, total_length, block_size)] 
      for k, t in concatenated_examples.items()
  }
  result['labels'] = result['input_ids'].copy()
  return result

In [41]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=preprocessing_num_workers,
    load_from_cache_file=not overwrite_cache,
    desc=f"Grouping texts in chunks of {block_size}"
)

Grouping texts in chunks of 512 #0:   0%|          | 0/16 [00:00<?, ?ba/s]

Grouping texts in chunks of 512 #2:   0%|          | 0/16 [00:00<?, ?ba/s]

Grouping texts in chunks of 512 #1:   0%|          | 0/16 [00:00<?, ?ba/s]

Grouping texts in chunks of 512 #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Grouping texts in chunks of 512 #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Grouping texts in chunks of 512 #1:   0%|          | 0/1 [00:00<?, ?ba/s]

In [42]:
import torch 
import random
import numpy as np

from transformers import GPT2Config, GPT2LMHeadModel


configuration = GPT2Config.from_pretrained(
    model_name,
    output_hidden_states=False
)

model = GPT2LMHeadModel.from_pretrained(
    model_name,
    config=configuration
)

model.resize_token_embeddings(len(tokenizer))

device = "cuda"
model.cuda()


seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [43]:
# model = torch.nn.parallel.DistributedDataParallel(model)

In [44]:
from transformers import (
    TrainingArguments,
    Trainer,
    default_data_collator
)

train_dataset = lm_datasets['train']
eval_dataset = lm_datasets['validation']

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    evaluation_strategy="no",
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    compute_metrics=None,
    preprocess_logits_for_metrics=None,
)

In [45]:
train_result = trainer.train(resume_from_checkpoint=None)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,3.867900
1000,3.221200
1500,3.193700
2000,3.182900
2500,3.167100
3000,3.167100
3500,3.156800
4000,3.153400
4500,3.139300
5000,3.103200


In [46]:
print("Saving model to %s" % output_dir)

trainer.save_model(output_dir=output_dir)
metrics = train_result.metrics
trainer.save_metrics("train", metrics)
trainer.save_state()

Saving model to model_gpt2


In [49]:
!zip -r model.zip /kaggle/working

updating: kaggle/working/ (stored 0%)
updating: kaggle/working/model_gpt2/ (stored 0%)
updating: kaggle/working/model_gpt2/training_args.bin (deflated 49%)
updating: kaggle/working/model_gpt2/added_tokens.json (deflated 20%)
updating: kaggle/working/model_gpt2/special_tokens_map.json (deflated 78%)
updating: kaggle/working/model_gpt2/runs/ (stored 0%)
updating: kaggle/working/model_gpt2/runs/Aug24_09-21-09_472164ef4c83/ (stored 0%)
updating: kaggle/working/model_gpt2/runs/Aug24_09-21-09_472164ef4c83/events.out.tfevents.1692868869.472164ef4c83.28.0 (deflated 61%)
updating: kaggle/working/model_gpt2/all_results.json (deflated 37%)
updating: kaggle/working/model_gpt2/config.json (deflated 51%)
updating: kaggle/working/model_gpt2/vocab.json (deflated 68%)
updating: kaggle/working/model_gpt2/generation_config.json (deflated 24%)
updating: kaggle/working/model_gpt2/train_results.json (deflated 37%)
updating: kaggle/working/model_gpt2/merges.txt (deflated 53%)
updating: kaggle/working/model_g

In [50]:
from IPython.display import FileLink
FileLink('model.zip')

/kaggle/working/model.zip

In [51]:
kwargs = {
    "finetuned_from": model_name,
    "tasks": "text-generation"
}
trainer.create_model_card(**kwargs)

In [56]:
model.eval()

prompt = "Write an essay on Title: Future in ai with tags: 'Artificial Intelligence', 'Mental Health'"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(torch.device("cuda"))

print(generated)

sample_outputs = model.generate(
    generated,
    do_sample=True,
    top_k=50,
    max_length=500,
    top_p=0.95,
    num_return_sequences=5
)

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[16594,   281, 14268,   319, 11851,    25, 10898,   287,   257,    72,
           351, 15940,    25,   705,  8001,  9542,  9345,  3256,   705,    44,
          2470,  3893,     6]], device='cuda:0')
0: Write an essay on Title: Future in ai with tags: 'Artificial Intelligence', 'Mental Health', Future, Futureproof, Software 
Article Text:  Ai is the future in ai for i and k users, not only that in its current state, but it has matured from the past and has achieved the ability to analyze data, analyse data, measure data, and even interact with data.

According to many research studies, Ai is evolving from being only a tool that is used in ai to being a tool that can be used for different domains. In this post, I will explain the main reasons behind this thinking, how AI and AI-led technologies are impacting our lives, in the hopes of enhancing our knowledge and saving lives.

The aim is to show how data can help us think beyond traditional thinking, and how people are able to de

In [53]:
!pip install --quiet pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [54]:
output_dir = "model_gpt2"
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50259, bias=False)
)